# CAPSTONE PROJECT - PART 5
__Michael Gat__  
__General Assembly Santa Monica, Data Science Immersive, Summer 2016__

In this notebook, we'll build upon __Part 4__ by adding by adding all available features from the original dataset and letting the model run on all of them.

In [1]:
# IMPORT LIBRARIES ############################################################

import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler

#### READ IN DATA
We have a clean dataset from __Part 1__. We'll import that, then select only the columns we want to deal with at this time. This code is similar to __Part 4__, please review that notebook for additional details/comments.

In [2]:
df = pd.read_csv('diabetic_data_clean.csv')

In [3]:
df.columns

Index([u'race', u'gender', u'admission_type_id', u'discharge_disposition_id',
       u'admission_source_id', u'time_in_hospital', u'num_lab_procedures',
       u'num_procedures', u'num_medications', u'number_outpatient',
       u'number_emergency', u'number_inpatient', u'number_diagnoses',
       u'max_glu_serum', u'A1Cresult', u'metformin', u'repaglinide',
       u'nateglinide', u'chlorpropamide', u'glimepiride', u'acetohexamide',
       u'glipizide', u'glyburide', u'tolbutamide', u'pioglitazone',
       u'rosiglitazone', u'acarbose', u'miglitol', u'troglitazone',
       u'tolazamide', u'examide', u'citoglipton', u'insulin',
       u'glyburide-metformin', u'glipizide-metformin',
       u'glimepiride-pioglitazone', u'metformin-rosiglitazone',
       u'metformin-pioglitazone', u'change', u'diabetesMed', u'age_group',
       u'readmit', u'428', u'276', u'414', u'401', u'427', u'599', u'496',
       u'403', u'486', u'786', u'780', u'491', u'410', u'682', u'584', u'585',
       u'707', u'5

In [4]:
# Include all possible predictor columns in the X set.

X = df.ix[:,0:41]
X = X.join(df.ix[:,42:62])
y = df['readmit']
X.columns

Index([u'race', u'gender', u'admission_type_id', u'discharge_disposition_id',
       u'admission_source_id', u'time_in_hospital', u'num_lab_procedures',
       u'num_procedures', u'num_medications', u'number_outpatient',
       u'number_emergency', u'number_inpatient', u'number_diagnoses',
       u'max_glu_serum', u'A1Cresult', u'metformin', u'repaglinide',
       u'nateglinide', u'chlorpropamide', u'glimepiride', u'acetohexamide',
       u'glipizide', u'glyburide', u'tolbutamide', u'pioglitazone',
       u'rosiglitazone', u'acarbose', u'miglitol', u'troglitazone',
       u'tolazamide', u'examide', u'citoglipton', u'insulin',
       u'glyburide-metformin', u'glipizide-metformin',
       u'glimepiride-pioglitazone', u'metformin-rosiglitazone',
       u'metformin-pioglitazone', u'change', u'diabetesMed', u'age_group',
       u'428', u'276', u'414', u'401', u'427', u'599', u'496', u'403', u'486',
       u'786', u'780', u'491', u'410', u'682', u'584', u'585', u'707', u'518',
       u'411',

## BUILD FEATURE SELECTION
Still working with the recommended Chi2, test for different numbers of features, using code adapted from __Part2__.

In [5]:
X_std = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.75)

### Undersampling
As in the previous section

In [6]:
# Use random under sampler module to reduce the number of "no readmit" records influencing the learning
rus = RandomUnderSampler()
X_resampled, y_resampled = rus.fit_sample(X, y)

### Naive Bayes Only
To keep life simpler, only going to bother with Gaussian Naive Bayes for this run. Other classifiers haven't been useful.

In [7]:
clf = GaussianNB()

for numtest in range (3,20):
    ch2 = SelectKBest(chi2, k=numtest)
    X_train_fit = ch2.fit_transform(X_resampled, y_resampled)
    print 'SELECTED FEATURES:' + str(numtest)
    col_indices = ch2.get_support(indices=True)
    for i in col_indices:
        print X_train.columns.values[i]
    print ch2.scores_
    print
    
    X_test_xform = ch2.transform(X_test)
    
    clf.fit(X_train_fit, y_resampled)
    score = clf.score(X_test_xform, y_test)
    y_pred = clf.predict(X_test_xform)
    y_pred = clf.predict(X_test_xform)
    cm = confusion_matrix(y_test, y_pred)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    print clf
    print score
    print cm
    print "AUC Metrics:"
    print auc(fpr, tpr)
    print

SELECTED FEATURES:3
discharge_disposition_id
number_emergency
number_inpatient
[  6.10052622e-02   8.41203351e-01   7.02226182e+00   1.05018888e+03
   2.23089307e+00   2.58750785e+02   2.97070383e+02   2.39607562e+01
   3.53839628e+02   6.09747167e+01   6.50983007e+02   2.60849744e+03
   6.39902930e+01   9.30275229e+00   5.55626496e+00   6.41668620e+01
   1.32625995e+01   3.65558912e-01   7.11111111e+00   1.68752834e+00
              nan   1.77872711e+00   9.17985012e+00   2.00000000e+00
   4.30284302e+00   5.46799431e+00   4.04958678e-01   0.00000000e+00
   2.00000000e+00   2.00000000e+00              nan              nan
   4.77954783e+01   3.17492711e+00   0.00000000e+00              nan
   2.00000000e+00              nan   1.07275427e+01   5.79240223e+00
   5.16951511e+00   6.17959784e+01   5.24306473e+00   2.87032923e+01
   8.48810988e+01   2.67111853e-02   3.18388564e-02   2.32903226e+00
   5.75010047e+01   5.08450704e+00   4.45454545e+01   5.19902439e+00
   7.39882698e+00   3.88

## MISSING  
### K-folds validation
The results are fairly consistent over multiple runs, but should formalize this.  

### Add additional features?
Unlikely that any additional diagnostic codes would matter given how little most of them appear in the data, but could increase the number from "top 20" to "top 50" and see what happens. Beyond that the codes are mostly one-off diagnoses.